In [1]:
# Range dependent Hamiltonian Algorithm for numerical QUBO formulation

# This code only use one subdomain because of time limit of D-Wave account
# For a genearal code, please use additional "for" loop in "cef2"

# 32 by 32 matrix A
# Range of x_i = (-128, 127)
# Subrange of x_i = (0, 3) = q_(i,1) + 2q(i,2)

import numpy as np
import random, math
import copy
import sys
import math

Dimension = 16
A = np.random.randint(-10, 10, size=(Dimension,Dimension))
np.set_printoptions(threshold=sys.maxsize)
print("Matrix A:")
print(A)

x_fin = np.random.randint(-128, 127, size=Dimension)
print("\nSolution vector x:")
print(x_fin)

b = np.dot(A, x_fin)
print("\nVector b:")
print(b)

Matrix A:
[[ -9   3  -3   9   8  -8  -1  -9  -1   6  -8   5   6  -4 -10   8]
 [  4   3  -1   0   3   8   8 -10   3   4  -8   8  -2   5   7  -5]
 [-10   9   9  -5 -10  -7  -3   9   2   5   7  -9  -8   9   7   1]
 [ -8   5  -9  -6   2 -10   6  -1  -8  -4  -7   2  -2   9   1   9]
 [  2   7  -7   6   5  -1   3   4 -10   1   2  -6   1   1  -6   6]
 [  9  -5   6   9  -8  -5  -4  -8   9   2  -4   5  -1  -6  -7   3]
 [ -2  -7   5   3  -7   6   8   1  -9  -6   4  -9   9  -5   0   2]
 [ -1  -5   3  -6  -4   3  -3  -9   2   3  -4  -6   3   6   5   8]
 [  2   4   9  -7  -3  -4  -3   5   8  -4   7   3  -5   1  -5  -8]
 [  2   6  -7  -1   6  -3  -1  -9   2   9  -4   5  -3  -6  -9   4]
 [ -3   7   2  -2   9  -5  -4  -9  -3   0 -10   1  -7  -7  -9   8]
 [ -4   0  -7 -10   5   5   8  -2   5   0   1  -6   6  -1   6   5]
 [  4 -10 -10  -6   2   1   7   5  -6   4   2   6   8   5  -3  -1]
 [ -7   1   9  -4   5   3  -2   4   6  -3   9   8  -8  -6   3  -2]
 [ -4   8 -10  -2 -10   6   6   9  -3   5  -2  -6  -

In [2]:
qubits = 2
# Calculate translation number including solution vector x
Trs = np.zeros(Dimension)
x_qubit = np.zeros(Dimension)
sub_size = pow(2,qubits)
for k in range(Dimension):
    Val = divmod(x_fin[k], sub_size)
    Trs[k] = sub_size*Val[0]
    x_qubit[k] = Val[1]
print("Translation vector T:")
print(Trs)
print("\nQubit notation:")
print(x_qubit)
c = b - np.dot(A, Trs)
print("\nUpdated vector c:")
print(c)

Translation vector T:
[ 116. -120.   40.    8.  -68.  -52.  -84.  -96.  -68.  -60.    0.   44.
  104.  120.   88.   92.]

Qubit notation:
[2. 3. 0. 3. 3. 0. 1. 1. 2. 1. 0. 1. 1. 1. 1. 3.]

Updated vector c:
[ 57.  37. -21.   9.  54.  14. -51. -20. -30.  39.  19.  13. -17.  -6.
  -6.  -1.]


In [3]:
QM = np.zeros((qubits*Dimension, qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*l)*pow(A[k][i],2)
            cef2 = pow(2,l+1)*A[k][i]*c[k]
            po = qubits*i + l
            QM[po][po] = QM[po][po] + cef1 - cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, l1+l2+1)*pow(A[k][i],2)
                po1 = qubits*i + l1
                po2 = qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, l1+l2+1)*A[k][i]*A[k][j] 
                    po1 = qubits*i + l1
                    po2 = qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef

# Print Matrix Q
print("# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n")

# Print Python code for the run in D-Wave quantum processing unit
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(qubits*Dimension-1): 
    linear = i + 1
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",qubits*Dimension,"','q",qubits*Dimension,"'):",format(QM[qubits*Dimension-1][qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(qubits*Dimension-1):
    for j in range(i+1,qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = i + 1
            qdrt2 = j + 1
            if i == qubits*Dimension-2 and j == qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")      

# Matrix Q is
[[ 4.5600e+02  2.1840e+03 -6.3600e+02 -1.2720e+03 -8.2000e+01 -1.6400e+02
   4.1200e+02  8.2400e+02 -2.5800e+02 -5.1600e+02  1.3800e+02  2.7600e+02
  -6.0000e+01 -1.2000e+02 -5.0000e+01 -1.0000e+02  2.3000e+02  4.6000e+02
   5.4000e+01  1.0800e+02 -6.8000e+01 -1.3600e+02  3.3000e+02  6.6000e+02
   2.5000e+02  5.0000e+02 -3.0200e+02 -6.0400e+02 -2.0200e+02 -4.0400e+02
  -3.7600e+02 -7.5200e+02]
 [ 0.0000e+00  2.0040e+03 -1.2720e+03 -2.5440e+03 -1.6400e+02 -3.2800e+02
   8.2400e+02  1.6480e+03 -5.1600e+02 -1.0320e+03  2.7600e+02  5.5200e+02
  -1.2000e+02 -2.4000e+02 -1.0000e+02 -2.0000e+02  4.6000e+02  9.2000e+02
   1.0800e+02  2.1600e+02 -1.3600e+02 -2.7200e+02  6.6000e+02  1.3200e+03
   5.0000e+02  1.0000e+03 -6.0400e+02 -1.2080e+03 -4.0400e+02 -8.0800e+02
  -7.5200e+02 -1.5040e+03]
 [ 0.0000e+00  0.0000e+00 -1.8950e+03  2.4760e+03 -6.4000e+01 -1.2800e+02
  -2.0800e+02 -4.1600e+02  3.4000e+02  6.8000e+02 -2.3400e+02 -4.6800e+02
  -9.2000e+01 -1.8400e+02 -7.2000e+01 -1.440

In [8]:
from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):456.0, ('q2','q2'):2004.0, ('q3','q3'):-1895.0, ('q4','q4'):-2552.0, ('q5','q5'):3793.0, ('q6','q6'):9128.0, ('q7','q7'):-1706.0, ('q8','q8'):-2312.0, ('q9','q9'):-3182.0, ('q10','q10'):-4980.0, ('q11','q11'):1914.0, ('q12','q12'):4864.0, ('q13','q13'):287.0, ('q14','q14'):1348.0, ('q15','q15'):4236.0, ('q16','q16'):10148.0, ('q17','q17'):777.0, ('q18','q18'):2616.0, ('q19','q19'):-1843.0, ('q20','q20'):-3088.0, ('q21','q21'):3842.0, ('q22','q22'):8752.0, ('q23','q23'):-1531.0, ('q24','q24'):-1936.0, ('q25','q25'):757.0, ('q26','q26'):2544.0, ('q27','q27'):1592.0, ('q28','q28'):4204.0, ('q29','q29'):3058.0, ('q30','q30'):7236.0, ('q31','q31'):-1669.0, ('q32','q32'):-2412.0}

quadratic = {('q1','q2'):2184.0, ('q1','q3'):-636.0, ('q1','q4'):-1272.0, ('q1','q5'):-82.0, ('q1','q6'):-164.0, ('q1','q7'):412.0, ('q1','q8'):824.0, ('q1','q9'):-258.0, ('q1','q10'):-516.0, ('q1','q11'):138.0, ('q1','q12'):276.0, ('q1','q13'):-60.0, ('q1','q14'):-120.0, ('q1','q15'):-50.0, ('q1','q16'):-100.0, ('q1','q17'):230.0, ('q1','q18'):460.0, ('q1','q19'):54.0, ('q1','q20'):108.0, ('q1','q21'):-68.0, ('q1','q22'):-136.0, ('q1','q23'):330.0, ('q1','q24'):660.0, ('q1','q25'):250.0, ('q1','q26'):500.0, ('q1','q27'):-302.0, ('q1','q28'):-604.0, ('q1','q29'):-202.0, ('q1','q30'):-404.0, ('q1','q31'):-376.0, ('q1','q32'):-752.0, ('q2','q3'):-1272.0, ('q2','q4'):-2544.0, ('q2','q5'):-164.0, ('q2','q6'):-328.0, ('q2','q7'):824.0, ('q2','q8'):1648.0, ('q2','q9'):-516.0, ('q2','q10'):-1032.0, ('q2','q11'):276.0, ('q2','q12'):552.0, ('q2','q13'):-120.0, ('q2','q14'):-240.0, ('q2','q15'):-100.0, ('q2','q16'):-200.0, ('q2','q17'):460.0, ('q2','q18'):920.0, ('q2','q19'):108.0, ('q2','q20'):216.0, ('q2','q21'):-136.0, ('q2','q22'):-272.0, ('q2','q23'):660.0, ('q2','q24'):1320.0, ('q2','q25'):500.0, ('q2','q26'):1000.0, ('q2','q27'):-604.0, ('q2','q28'):-1208.0, ('q2','q29'):-404.0, ('q2','q30'):-808.0, ('q2','q31'):-752.0, ('q2','q32'):-1504.0, ('q3','q4'):2476.0, ('q3','q5'):-64.0, ('q3','q6'):-128.0, ('q3','q7'):-208.0, ('q3','q8'):-416.0, ('q3','q9'):340.0, ('q3','q10'):680.0, ('q3','q11'):-234.0, ('q3','q12'):-468.0, ('q3','q13'):-92.0, ('q3','q14'):-184.0, ('q3','q15'):-72.0, ('q3','q16'):-144.0, ('q3','q17'):-62.0, ('q3','q18'):-124.0, ('q3','q19'):174.0, ('q3','q20'):348.0, ('q3','q21'):-156.0, ('q3','q22'):-312.0, ('q3','q23'):-150.0, ('q3','q24'):-300.0, ('q3','q25'):-786.0, ('q3','q26'):-1572.0, ('q3','q27'):256.0, ('q3','q28'):512.0, ('q3','q29'):-124.0, ('q3','q30'):-248.0, ('q3','q31'):198.0, ('q3','q32'):396.0, ('q4','q5'):-128.0, ('q4','q6'):-256.0, ('q4','q7'):-416.0, ('q4','q8'):-832.0, ('q4','q9'):680.0, ('q4','q10'):1360.0, ('q4','q11'):-468.0, ('q4','q12'):-936.0, ('q4','q13'):-184.0, ('q4','q14'):-368.0, ('q4','q15'):-144.0, ('q4','q16'):-288.0, ('q4','q17'):-124.0, ('q4','q18'):-248.0, ('q4','q19'):348.0, ('q4','q20'):696.0, ('q4','q21'):-312.0, ('q4','q22'):-624.0, ('q4','q23'):-300.0, ('q4','q24'):-600.0, ('q4','q25'):-1572.0, ('q4','q26'):-3144.0, ('q4','q27'):512.0, ('q4','q28'):1024.0, ('q4','q29'):-248.0, ('q4','q30'):-496.0, ('q4','q31'):396.0, ('q4','q32'):792.0, ('q5','q6'):3084.0, ('q5','q7'):42.0, ('q5','q8'):84.0, ('q5','q9'):-380.0, ('q5','q10'):-760.0, ('q5','q11'):-32.0, ('q5','q12'):-64.0, ('q5','q13'):-664.0, ('q5','q14'):-1328.0, ('q5','q15'):-22.0, ('q5','q16'):-44.0, ('q5','q17'):656.0, ('q5','q18'):1312.0, ('q5','q19'):-370.0, ('q5','q20'):-740.0, ('q5','q21'):554.0, ('q5','q22'):1108.0, ('q5','q23'):-26.0, ('q5','q24'):-52.0, ('q5','q25'):-378.0, ('q5','q26'):-756.0, ('q5','q27'):-258.0, ('q5','q28'):-516.0, ('q5','q29'):146.0, ('q5','q30'):292.0, ('q5','q31'):-464.0, ('q5','q32'):-928.0, ('q6','q7'):84.0, ('q6','q8'):168.0, ('q6','q9'):-760.0, ('q6','q10'):-1520.0, ('q6','q11'):-64.0, ('q6','q12'):-128.0, ('q6','q13'):-1328.0, ('q6','q14'):-2656.0, ('q6','q15'):-44.0, ('q6','q16'):-88.0, ('q6','q17'):1312.0, ('q6','q18'):2624.0, ('q6','q19'):-740.0, ('q6','q20'):-1480.0, ('q6','q21'):1108.0, ('q6','q22'):2216.0, ('q6','q23'):-52.0, ('q6','q24'):-104.0, ('q6','q25'):-756.0, ('q6','q26'):-1512.0, ('q6','q27'):-516.0, ('q6','q28'):-1032.0, ('q6','q29'):292.0, ('q6','q30'):584.0, ('q6','q31'):-928.0, ('q6','q32'):-1856.0, ('q7','q8'):2200.0, ('q7','q9'):-96.0, ('q7','q10'):-192.0, ('q7','q11'):-218.0, ('q7','q12'):-436.0, ('q7','q13'):-204.0, ('q7','q14'):-408.0, ('q7','q15'):-206.0, ('q7','q16'):-412.0, ('q7','q17'):-122.0, ('q7','q18'):-244.0, ('q7','q19'):112.0, ('q7','q20'):224.0, ('q7','q21'):-276.0, ('q7','q22'):-552.0, ('q7','q23'):168.0, ('q7','q24'):336.0, ('q7','q25'):232.0, ('q7','q26'):464.0, ('q7','q27'):-526.0, ('q7','q28'):-1052.0, ('q7','q29'):-504.0, ('q7','q30'):-1008.0, ('q7','q31'):72.0, ('q7','q32'):144.0, ('q8','q9'):-192.0, ('q8','q10'):-384.0, ('q8','q11'):-436.0, ('q8','q12'):-872.0, ('q8','q13'):-408.0, ('q8','q14'):-816.0, ('q8','q15'):-412.0, ('q8','q16'):-824.0, ('q8','q17'):-244.0, ('q8','q18'):-488.0, ('q8','q19'):224.0, ('q8','q20'):448.0, ('q8','q21'):-552.0, ('q8','q22'):-1104.0, ('q8','q23'):336.0, ('q8','q24'):672.0, ('q8','q25'):464.0, ('q8','q26'):928.0, ('q8','q27'):-1052.0, ('q8','q28'):-2104.0, ('q8','q29'):-1008.0, ('q8','q30'):-2016.0, ('q8','q31'):144.0, ('q8','q32'):288.0, ('q9','q10'):2768.0, ('q9','q11'):-30.0, ('q9','q12'):-60.0, ('q9','q13'):24.0, ('q9','q14'):48.0, ('q9','q15'):-782.0, ('q9','q16'):-1564.0, ('q9','q17'):-172.0, ('q9','q18'):-344.0, ('q9','q19'):6.0, ('q9','q20'):12.0, ('q9','q21'):-388.0, ('q9','q22'):-776.0, ('q9','q23'):538.0, ('q9','q24'):1076.0, ('q9','q25'):116.0, ('q9','q26'):232.0, ('q9','q27'):-296.0, ('q9','q28'):-592.0, ('q9','q29'):-482.0, ('q9','q30'):-964.0, ('q9','q31'):242.0, ('q9','q32'):484.0, ('q10','q11'):-60.0, ('q10','q12'):-120.0, ('q10','q13'):48.0, ('q10','q14'):96.0, ('q10','q15'):-1564.0, ('q10','q16'):-3128.0, ('q10','q17'):-344.0, ('q10','q18'):-688.0, ('q10','q19'):12.0, ('q10','q20'):24.0, ('q10','q21'):-776.0, ('q10','q22'):-1552.0, ('q10','q23'):1076.0, ('q10','q24'):2152.0, ('q10','q25'):232.0, ('q10','q26'):464.0, ('q10','q27'):-592.0, ('q10','q28'):-1184.0, ('q10','q29'):-964.0, ('q10','q30'):-1928.0, ('q10','q31'):484.0, ('q10','q32'):968.0, ('q11','q12'):2072.0, ('q11','q13'):402.0, ('q11','q14'):804.0, ('q11','q15'):-6.0, ('q11','q16'):-12.0, ('q11','q17'):-6.0, ('q11','q18'):-12.0, ('q11','q19'):-112.0, ('q11','q20'):-224.0, ('q11','q21'):228.0, ('q11','q22'):456.0, ('q11','q23'):-212.0, ('q11','q24'):-424.0, ('q11','q25'):190.0, ('q11','q26'):380.0, ('q11','q27'):78.0, ('q11','q28'):156.0, ('q11','q29'):544.0, ('q11','q30'):1088.0, ('q11','q31'):-366.0, ('q11','q32'):-732.0, ('q12','q13'):804.0, ('q12','q14'):1608.0, ('q12','q15'):-12.0, ('q12','q16'):-24.0, ('q12','q17'):-12.0, ('q12','q18'):-24.0, ('q12','q19'):-224.0, ('q12','q20'):-448.0, ('q12','q21'):456.0, ('q12','q22'):912.0, ('q12','q23'):-424.0, ('q12','q24'):-848.0, ('q12','q25'):380.0, ('q12','q26'):760.0, ('q12','q27'):156.0, ('q12','q28'):312.0, ('q12','q29'):1088.0, ('q12','q30'):2176.0, ('q12','q31'):-732.0, ('q12','q32'):-1464.0, ('q13','q14'):1548.0, ('q13','q15'):140.0, ('q13','q16'):280.0, ('q13','q17'):-438.0, ('q13','q18'):-876.0, ('q13','q19'):-16.0, ('q13','q20'):-32.0, ('q13','q21'):-76.0, ('q13','q22'):-152.0, ('q13','q23'):-140.0, ('q13','q24'):-280.0, ('q13','q25'):356.0, ('q13','q26'):712.0, ('q13','q27'):280.0, ('q13','q28'):560.0, ('q13','q29'):290.0, ('q13','q30'):580.0, ('q13','q31'):76.0, ('q13','q32'):152.0, ('q14','q15'):280.0, ('q14','q16'):560.0, ('q14','q17'):-876.0, ('q14','q18'):-1752.0, ('q14','q19'):-32.0, ('q14','q20'):-64.0, ('q14','q21'):-152.0, ('q14','q22'):-304.0, ('q14','q23'):-280.0, ('q14','q24'):-560.0, ('q14','q25'):712.0, ('q14','q26'):1424.0, ('q14','q27'):560.0, ('q14','q28'):1120.0, ('q14','q29'):580.0, ('q14','q30'):1160.0, ('q14','q31'):152.0, ('q14','q32'):304.0, ('q15','q16'):3352.0, ('q15','q17'):-216.0, ('q15','q18'):-432.0, ('q15','q19'):-216.0, ('q15','q20'):-432.0, ('q15','q21'):958.0, ('q15','q22'):1916.0, ('q15','q23'):-452.0, ('q15','q24'):-904.0, ('q15','q25'):-202.0, ('q15','q26'):-404.0, ('q15','q27'):322.0, ('q15','q28'):644.0, ('q15','q29'):456.0, ('q15','q30'):912.0, ('q15','q31'):-470.0, ('q15','q32'):-940.0, ('q16','q17'):-432.0, ('q16','q18'):-864.0, ('q16','q19'):-432.0, ('q16','q20'):-864.0, ('q16','q21'):1916.0, ('q16','q22'):3832.0, ('q16','q23'):-904.0, ('q16','q24'):-1808.0, ('q16','q25'):-404.0, ('q16','q26'):-808.0, ('q16','q27'):644.0, ('q16','q28'):1288.0, ('q16','q29'):912.0, ('q16','q30'):1824.0, ('q16','q31'):-940.0, ('q16','q32'):-1880.0, ('q17','q18'):2124.0, ('q17','q19'):102.0, ('q17','q20'):204.0, ('q17','q21'):166.0, ('q17','q22'):332.0, ('q17','q23'):388.0, ('q17','q24'):776.0, ('q17','q25'):-338.0, ('q17','q26'):-676.0, ('q17','q27'):-246.0, ('q17','q28'):-492.0, ('q17','q29'):144.0, ('q17','q30'):288.0, ('q17','q31'):-386.0, ('q17','q32'):-772.0, ('q18','q19'):204.0, ('q18','q20'):408.0, ('q18','q21'):332.0, ('q18','q22'):664.0, ('q18','q23'):776.0, ('q18','q24'):1552.0, ('q18','q25'):-676.0, ('q18','q26'):-1352.0, ('q18','q27'):-492.0, ('q18','q28'):-984.0, ('q18','q29'):288.0, ('q18','q30'):576.0, ('q18','q31'):-772.0, ('q18','q32'):-1544.0, ('q19','q20'):1196.0, ('q19','q21'):-310.0, ('q19','q22'):-620.0, ('q19','q23'):116.0, ('q19','q24'):232.0, ('q19','q25'):-70.0, ('q19','q26'):-140.0, ('q19','q27'):58.0, ('q19','q28'):116.0, ('q19','q29'):-140.0, ('q19','q30'):-280.0, ('q19','q31'):208.0, ('q19','q32'):416.0, ('q20','q21'):-620.0, ('q20','q22'):-1240.0, ('q20','q23'):232.0, ('q20','q24'):464.0, ('q20','q25'):-140.0, ('q20','q26'):-280.0, ('q20','q27'):116.0, ('q20','q28'):232.0, ('q20','q29'):-280.0, ('q20','q30'):-560.0, ('q20','q31'):416.0, ('q20','q32'):832.0, ('q21','q22'):2136.0, ('q21','q23'):-292.0, ('q21','q24'):-584.0, ('q21','q25'):-66.0, ('q21','q26'):-132.0, ('q21','q27'):52.0, ('q21','q28'):104.0, ('q21','q29'):346.0, ('q21','q30'):692.0, ('q21','q31'):-552.0, ('q21','q32'):-1104.0, ('q22','q23'):-584.0, ('q22','q24'):-1168.0, ('q22','q25'):-132.0, ('q22','q26'):-264.0, ('q22','q27'):104.0, ('q22','q28'):208.0, ('q22','q29'):692.0, ('q22','q30'):1384.0, ('q22','q31'):-1104.0, ('q22','q32'):-2208.0, ('q23','q24'):2252.0, ('q23','q25'):-130.0, ('q23','q26'):-260.0, ('q23','q27'):-316.0, ('q23','q28'):-632.0, ('q23','q29'):-398.0, ('q23','q30'):-796.0, ('q23','q31'):-272.0, ('q23','q32'):-544.0, ('q24','q25'):-260.0, ('q24','q26'):-520.0, ('q24','q27'):-632.0, ('q24','q28'):-1264.0, ('q24','q29'):-796.0, ('q24','q30'):-1592.0, ('q24','q31'):-544.0, ('q24','q32'):-1088.0, ('q25','q26'):2060.0, ('q25','q27'):-104.0, ('q25','q28'):-208.0, ('q25','q29'):-70.0, ('q25','q30'):-140.0, ('q25','q31'):146.0, ('q25','q32'):292.0, ('q26','q27'):-208.0, ('q26','q28'):-416.0, ('q26','q29'):-140.0, ('q26','q30'):-280.0, ('q26','q31'):292.0, ('q26','q32'):584.0, ('q27','q28'):2040.0, ('q27','q29'):590.0, ('q27','q30'):1180.0, ('q27','q31'):-46.0, ('q27','q32'):-92.0, ('q28','q29'):1180.0, ('q28','q30'):2360.0, ('q28','q31'):-92.0, ('q28','q32'):-184.0, ('q29','q30'):2240.0, ('q29','q31'):-300.0, ('q29','q32'):-600.0, ('q30','q31'):-600.0, ('q30','q32'):-1200.0, ('q31','q32'):1852.0}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=10000)
print(sampleset)

     q1 q10 q11 q12 q13 q14 q15 q16 q17 q18 q19 q2 ... q9   energy num_oc. ...
0     0   1   0   0   1   0   1   0   0   1   0  1 ...  1 -14267.0       1 ...
3568  0   1   0   0   1   0   1   0   0   1   0  1 ...  1 -14267.0       1 ...
3     1   1   0   0   1   0   1   0   0   1   1  0 ...  1 -14058.0       1 ...
341   0   1   1   0   0   0   1   0   1   0   1  1 ...  1 -13964.0       1 ...
7     0   1   0   0   0   1   1   0   0   1   1  1 ...  1 -13948.0       1 ...
8     0   1   0   0   1   0   1   0   0   1   1  1 ...  1 -13947.0       1 ...
128   0   1   0   0   0   1   1   0   0   1   0  1 ...  1 -13916.0       1 ...
2020  0   1   0   0   0   0   1   0   0   1   1  1 ...  1 -13909.0       1 ...
759   0   1   0   0   0   1   1   0   0   1   0  1 ...  1 -13896.0       1 ...
11    1   1   0   0   1   0   1   0   0   1   0  0 ...  1 -13889.0       1 ...
198   0   1   0   0   0   0   1   0   1   0   0  1 ...  1 -13887.0       1 ...
13    0   1   0   0   0   0   1   0   1   0   0  1 .

In [9]:
sampleset = sampler_auto.sample_qubo(Q, num_reads=10000)
print(sampleset)

     q1 q10 q11 q12 q13 q14 q15 q16 q17 q18 q19 q2 ... q9   energy num_oc. ...
3     0   1   0   0   1   0   1   0   0   1   1  1 ...  1 -14084.0       1 ...
5     1   1   0   0   1   0   1   0   0   1   1  0 ...  1 -14058.0       1 ...
283   1   1   0   0   1   0   1   0   0   1   1  0 ...  1 -14058.0       1 ...
6     0   1   0   0   1   0   1   0   0   1   1  1 ...  1 -14032.0       1 ...
5279  1   1   0   0   1   0   1   0   0   1   0  0 ...  1 -13966.0       1 ...
11    1   1   0   0   0   1   1   0   0   1   0  0 ...  1 -13922.0       1 ...
65    1   1   0   0   0   1   1   0   0   1   0  0 ...  1 -13922.0       1 ...
13    1   1   0   0   1   0   1   0   0   1   1  0 ...  1 -13894.0       1 ...
16    0   1   0   0   1   0   1   0   0   1   1  1 ...  1 -13883.0       1 ...
17    1   1   0   0   0   1   1   0   1   1   1  0 ...  1 -13882.0       1 ...
18    0   1   0   0   1   0   1   0   0   1   0  1 ...  1 -13879.0       1 ...
20    1   1   1   0   1   0   1   0   0   1   1  0 .

In [10]:
sampleset = sampler_auto.sample_qubo(Q, num_reads=10000)
print(sampleset)

     q1 q10 q11 q12 q13 q14 q15 q16 q17 q18 q19 q2 ... q9   energy num_oc. ...
5183  0   1   0   0   1   0   1   0   0   1   1  1 ...  1 -14566.0       1 ...
14    0   1   0   0   1   0   1   0   0   1   0  1 ...  1 -14267.0       1 ...
0     0   1   0   0   1   0   1   0   1   0   1  1 ...  1 -14087.0       1 ...
1     0   1   0   0   0   1   1   0   1   1   1  1 ...  1 -14033.0       1 ...
2764  0   1   0   0   0   1   1   0   1   1   1  1 ...  1 -14006.0       1 ...
2     0   1   0   0   0   0   1   0   1   0   0  1 ...  1 -13991.0       1 ...
4     0   1   0   0   1   0   1   0   1   0   1  1 ...  1 -13958.0       1 ...
5     1   1   0   0   0   1   0   0   0   1   1  0 ...  1 -13955.0       1 ...
7874  0   1   0   0   1   0   1   0   0   1   1  1 ...  1 -13950.0       1 ...
6     0   1   0   0   1   0   1   0   1   1   0  1 ...  1 -13946.0       1 ...
7     0   1   0   0   1   0   1   0   1   1   1  1 ...  1 -13945.0       1 ...
8     0   1   0   0   1   0   1   0   1   0   1  1 .